# Sim2Real Transfer Analysis - GentleHumanoid

This notebook provides comprehensive visualization and analysis tools for debugging sim2real transfer issues on the Unitree G1 humanoid robot.

## Data Structure
- **Time**: Elapsed seconds (1 col)
- **Joint Position**: Actual joint angles, 29 DOF (29 cols)
- **Joint Velocity**: Actual joint velocities (29 cols)
- **Quaternion**: IMU orientation wxyz (4 cols)
- **Gyroscope**: Angular velocity (3 cols)
- **Motor Torque**: Estimated torques (29 cols)
- **Joint Target**: Commanded positions (29 cols)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.spatial.transform import Rotation
import warnings
warnings.filterwarnings('ignore')

# Plotting settings
plt.rcParams['figure.figsize'] = [14, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

## 1. Data Loading and Configuration

In [ ]:
# Joint names in real robot order (29 DOF)
JOINT_NAMES = [
    # Left leg (0-5)
    "left_hip_pitch", "left_hip_roll", "left_hip_yaw", 
    "left_knee", "left_ankle_pitch", "left_ankle_roll",
    # Right leg (6-11)
    "right_hip_pitch", "right_hip_roll", "right_hip_yaw", 
    "right_knee", "right_ankle_pitch", "right_ankle_roll",
    # Waist (12-14)
    "waist_yaw", "waist_roll", "waist_pitch",
    # Left arm (15-21)
    "left_shoulder_pitch", "left_shoulder_roll", "left_shoulder_yaw",
    "left_elbow", "left_wrist_roll", "left_wrist_pitch", "left_wrist_yaw",
    # Right arm (22-28)
    "right_shoulder_pitch", "right_shoulder_roll", "right_shoulder_yaw",
    "right_elbow", "right_wrist_roll", "right_wrist_pitch", "right_wrist_yaw"
]

# Joint groups for organized plotting
JOINT_GROUPS = {
    "Left Leg": list(range(0, 6)),
    "Right Leg": list(range(6, 12)),
    "Waist": list(range(12, 15)),
    "Left Arm": list(range(15, 22)),
    "Right Arm": list(range(22, 29)),
}

# Data column indices
IDX_TIME = 0
IDX_JOINT_POS = slice(1, 30)       # 29 cols
IDX_JOINT_VEL = slice(30, 59)      # 29 cols
IDX_QUAT = slice(59, 63)           # 4 cols (wxyz)
IDX_GYRO = slice(63, 66)           # 3 cols
IDX_TORQUE = slice(66, 95)         # 29 cols
IDX_TARGET = slice(95, 124)        # 29 cols

CONTROL_FREQ = 50  # Hz

In [ ]:
# Load data
DATA_PATH = "../eval_data/unitree_g1_data_log.csv"

raw_data = pd.read_csv(DATA_PATH, header=0)
data = raw_data.values

# Filter out zero rows (unfilled buffer entries)
valid_mask = data[:, IDX_TIME] > 0
data = data[valid_mask]

print(f"Loaded {len(data)} timesteps ({len(data)/CONTROL_FREQ:.1f} seconds)")
print(f"Data shape: {data.shape}")
print(f"Time range: {data[0, IDX_TIME]:.2f}s - {data[-1, IDX_TIME]:.2f}s")

In [ ]:
# Extract data columns
time = data[:, IDX_TIME]
joint_pos = data[:, IDX_JOINT_POS]
joint_vel = data[:, IDX_JOINT_VEL]
quat = data[:, IDX_QUAT]  # wxyz
gyro = data[:, IDX_GYRO]
torque = data[:, IDX_TORQUE]
target = data[:, IDX_TARGET]

# Compute tracking error
tracking_error = target - joint_pos

## 2. Overview Statistics

In [ ]:
def print_stats(name, arr, joint_names=None):
    """Print statistics for array."""
    print(f"\n=== {name} Statistics ===")
    if arr.ndim == 1:
        print(f"  Mean: {np.mean(arr):.4f}, Std: {np.std(arr):.4f}")
        print(f"  Min: {np.min(arr):.4f}, Max: {np.max(arr):.4f}")
    else:
        print(f"  Shape: {arr.shape}")
        print(f"  Mean per joint: min={np.mean(arr, axis=0).min():.4f}, max={np.mean(arr, axis=0).max():.4f}")
        print(f"  Std per joint:  min={np.std(arr, axis=0).min():.4f}, max={np.std(arr, axis=0).max():.4f}")
        
        # Find most active joints
        std_per_joint = np.std(arr, axis=0)
        top_indices = np.argsort(std_per_joint)[-5:][::-1]
        print(f"  Most variable joints:")
        for idx in top_indices:
            name_str = joint_names[idx] if joint_names else f"Joint {idx}"
            print(f"    {name_str}: std={std_per_joint[idx]:.4f}")

print_stats("Joint Position (rad)", joint_pos, JOINT_NAMES)
print_stats("Joint Velocity (rad/s)", joint_vel, JOINT_NAMES)
print_stats("Motor Torque (Nm)", torque, JOINT_NAMES)
print_stats("Tracking Error (rad)", tracking_error, JOINT_NAMES)

## 3. Tracking Error Analysis

**Key diagnostic for sim2real**: Large tracking errors indicate:
- Model mismatch (friction, inertia)
- Gain tuning issues
- External disturbances

In [ ]:
fig, axes = plt.subplots(len(JOINT_GROUPS), 1, figsize=(14, 12), sharex=True)
fig.suptitle('Tracking Error by Joint Group (Target - Actual)', fontsize=14)

for ax, (group_name, indices) in zip(axes, JOINT_GROUPS.items()):
    for idx in indices:
        ax.plot(time, tracking_error[:, idx], label=JOINT_NAMES[idx], alpha=0.8)
    ax.set_ylabel('Error (rad)')
    ax.set_title(group_name)
    ax.legend(loc='upper right', fontsize=8, ncol=2)
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)

axes[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

In [ ]:
# RMS tracking error per joint
rms_error = np.sqrt(np.mean(tracking_error**2, axis=0))
max_abs_error = np.max(np.abs(tracking_error), axis=0)

fig, ax = plt.subplots(figsize=(14, 5))
x = np.arange(len(JOINT_NAMES))
width = 0.35

bars1 = ax.bar(x - width/2, rms_error, width, label='RMS Error', color='steelblue')
bars2 = ax.bar(x + width/2, max_abs_error, width, label='Max Abs Error', color='coral')

ax.set_ylabel('Error (rad)')
ax.set_title('Tracking Error Summary per Joint')
ax.set_xticks(x)
ax.set_xticklabels(JOINT_NAMES, rotation=45, ha='right', fontsize=8)
ax.legend()
ax.axhline(y=0.1, color='r', linestyle='--', alpha=0.5, label='Threshold (0.1 rad)')

plt.tight_layout()
plt.show()

# Flag joints with high error
high_error_joints = np.where(rms_error > 0.05)[0]
if len(high_error_joints) > 0:
    print("\n[WARNING] Joints with high RMS tracking error (>0.05 rad):")
    for idx in high_error_joints:
        print(f"  {JOINT_NAMES[idx]}: RMS={rms_error[idx]:.4f}, Max={max_abs_error[idx]:.4f}")

## 4. Joint Position vs Target Comparison

In [ ]:
def plot_joint_tracking(joint_indices, title="Joint Tracking"):
    """Plot actual vs target positions for selected joints."""
    n_joints = len(joint_indices)
    fig, axes = plt.subplots(n_joints, 1, figsize=(14, 2.5*n_joints), sharex=True)
    if n_joints == 1:
        axes = [axes]
    
    fig.suptitle(title, fontsize=14)
    
    for ax, idx in zip(axes, joint_indices):
        ax.plot(time, target[:, idx], 'b-', label='Target', alpha=0.7)
        ax.plot(time, joint_pos[:, idx], 'r-', label='Actual', alpha=0.7)
        ax.fill_between(time, target[:, idx], joint_pos[:, idx], alpha=0.2, color='gray')
        ax.set_ylabel(f'{JOINT_NAMES[idx]}\n(rad)')
        ax.legend(loc='upper right')
    
    axes[-1].set_xlabel('Time (s)')
    plt.tight_layout()
    plt.show()

# Plot left leg joints
plot_joint_tracking(JOINT_GROUPS["Left Leg"], "Left Leg Tracking")

In [ ]:
# Plot arm joints (often most problematic for compliance)
plot_joint_tracking(JOINT_GROUPS["Left Arm"], "Left Arm Tracking")

## 5. Motor Torque Analysis

**Key indicators**:
- Saturation (hitting limits) suggests insufficient compliance or model mismatch
- Spikes indicate impact/disturbance events
- Oscillations suggest gain instability

In [ ]:
fig, axes = plt.subplots(len(JOINT_GROUPS), 1, figsize=(14, 12), sharex=True)
fig.suptitle('Motor Torque by Joint Group', fontsize=14)

for ax, (group_name, indices) in zip(axes, JOINT_GROUPS.items()):
    for idx in indices:
        ax.plot(time, torque[:, idx], label=JOINT_NAMES[idx], alpha=0.8)
    ax.set_ylabel('Torque (Nm)')
    ax.set_title(group_name)
    ax.legend(loc='upper right', fontsize=8, ncol=2)

axes[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

In [ ]:
# Torque statistics and saturation detection
torque_abs_max = np.max(np.abs(torque), axis=0)
torque_mean = np.mean(torque, axis=0)
torque_std = np.std(torque, axis=0)

fig, ax = plt.subplots(figsize=(14, 5))
x = np.arange(len(JOINT_NAMES))

ax.bar(x, torque_abs_max, color='coral', alpha=0.8, label='Max |Torque|')
ax.errorbar(x, np.abs(torque_mean), yerr=torque_std, fmt='ko', capsize=3, 
            label='Mean +/- Std', markersize=4)

ax.set_ylabel('Torque (Nm)')
ax.set_title('Torque Summary per Joint')
ax.set_xticks(x)
ax.set_xticklabels(JOINT_NAMES, rotation=45, ha='right', fontsize=8)
ax.legend()

plt.tight_layout()
plt.show()

# Identify high torque joints
print("\nTop 5 joints by max torque:")
top_torque_idx = np.argsort(torque_abs_max)[-5:][::-1]
for idx in top_torque_idx:
    print(f"  {JOINT_NAMES[idx]}: max={torque_abs_max[idx]:.2f} Nm, mean={torque_mean[idx]:.2f}, std={torque_std[idx]:.2f}")

In [ ]:
# Torque spike detection (potential impacts or instability)
def detect_torque_spikes(torque_signal, threshold_std=3.0):
    """Detect spikes in torque signal using rolling statistics."""
    diff = np.abs(np.diff(torque_signal, axis=0))
    threshold = np.mean(diff) + threshold_std * np.std(diff)
    spike_mask = diff > threshold
    return spike_mask

spike_counts = np.zeros(len(JOINT_NAMES))
for j in range(len(JOINT_NAMES)):
    spikes = detect_torque_spikes(torque[:, j])
    spike_counts[j] = np.sum(spikes)

if np.any(spike_counts > 10):
    print("\n[WARNING] Joints with frequent torque spikes (>10):")
    for idx in np.where(spike_counts > 10)[0]:
        print(f"  {JOINT_NAMES[idx]}: {int(spike_counts[idx])} spikes")

## 6. IMU / Orientation Analysis

**Key checks**:
- Quaternion should remain normalized
- Large gyro readings indicate fast rotation or instability
- Drift in orientation suggests IMU issues

In [ ]:
# Quaternion analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Quaternion components
ax = axes[0, 0]
ax.plot(time, quat[:, 0], label='w')
ax.plot(time, quat[:, 1], label='x')
ax.plot(time, quat[:, 2], label='y')
ax.plot(time, quat[:, 3], label='z')
ax.set_title('Quaternion Components (wxyz)')
ax.set_xlabel('Time (s)')
ax.legend()

# Quaternion norm (should be ~1.0)
quat_norm = np.linalg.norm(quat, axis=1)
ax = axes[0, 1]
ax.plot(time, quat_norm)
ax.axhline(y=1.0, color='r', linestyle='--', alpha=0.5)
ax.set_title('Quaternion Norm (should be 1.0)')
ax.set_xlabel('Time (s)')
ax.set_ylim([0.95, 1.05])

# Gyroscope
ax = axes[1, 0]
ax.plot(time, gyro[:, 0], label='X (roll rate)')
ax.plot(time, gyro[:, 1], label='Y (pitch rate)')
ax.plot(time, gyro[:, 2], label='Z (yaw rate)')
ax.set_title('Gyroscope (rad/s)')
ax.set_xlabel('Time (s)')
ax.legend()

# Convert to Euler angles for intuition
try:
    # scipy expects xyzw, our data is wxyz
    quat_xyzw = quat[:, [1, 2, 3, 0]]
    euler = Rotation.from_quat(quat_xyzw).as_euler('xyz', degrees=True)
    ax = axes[1, 1]
    ax.plot(time, euler[:, 0], label='Roll')
    ax.plot(time, euler[:, 1], label='Pitch')
    ax.plot(time, euler[:, 2], label='Yaw')
    ax.set_title('Euler Angles (degrees)')
    ax.set_xlabel('Time (s)')
    ax.legend()
except Exception as e:
    axes[1, 1].text(0.5, 0.5, f'Euler conversion failed: {e}', 
                    ha='center', va='center', transform=axes[1, 1].transAxes)

plt.tight_layout()
plt.show()

print(f"\nQuaternion norm: mean={np.mean(quat_norm):.6f}, std={np.std(quat_norm):.6f}")
print(f"Gyro range: X=[{gyro[:,0].min():.2f}, {gyro[:,0].max():.2f}], "
      f"Y=[{gyro[:,1].min():.2f}, {gyro[:,1].max():.2f}], "
      f"Z=[{gyro[:,2].min():.2f}, {gyro[:,2].max():.2f}] rad/s")

## 7. Joint Velocity Analysis

**Key checks**:
- Excessive noise indicates sensor issues or vibration
- Velocity should correlate with position changes

In [ ]:
# Velocity statistics
vel_std = np.std(joint_vel, axis=0)
vel_max = np.max(np.abs(joint_vel), axis=0)

fig, ax = plt.subplots(figsize=(14, 5))
x = np.arange(len(JOINT_NAMES))
width = 0.35

ax.bar(x - width/2, vel_std, width, label='Std Dev', color='steelblue')
ax.bar(x + width/2, vel_max, width, label='Max |Velocity|', color='coral')

ax.set_ylabel('Velocity (rad/s)')
ax.set_title('Joint Velocity Statistics')
ax.set_xticks(x)
ax.set_xticklabels(JOINT_NAMES, rotation=45, ha='right', fontsize=8)
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Position-Velocity consistency check
# Numerical derivative of position should roughly match velocity
dt = 1.0 / CONTROL_FREQ
pos_deriv = np.diff(joint_pos, axis=0) / dt

# Compare for a few joints
test_joints = [0, 3, 15]  # hip_pitch, knee, shoulder_pitch

fig, axes = plt.subplots(len(test_joints), 1, figsize=(14, 3*len(test_joints)), sharex=True)
fig.suptitle('Velocity Consistency Check (measured vs numerical derivative)', fontsize=14)

for ax, idx in zip(axes, test_joints):
    ax.plot(time[1:], joint_vel[1:, idx], 'b-', label='Measured velocity', alpha=0.7)
    ax.plot(time[1:], pos_deriv[:, idx], 'r--', label='Position derivative', alpha=0.7)
    ax.set_ylabel(f'{JOINT_NAMES[idx]}\n(rad/s)')
    ax.legend(loc='upper right')

axes[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

## 8. Spectral Analysis (Oscillation Detection)

**Key checks**:
- Peaks at specific frequencies indicate oscillations
- High-frequency content suggests instability or noise

In [ ]:
def compute_fft(signal_data, fs=CONTROL_FREQ):
    """Compute FFT for a signal."""
    n = len(signal_data)
    fft_result = np.fft.rfft(signal_data - np.mean(signal_data))
    freqs = np.fft.rfftfreq(n, 1/fs)
    magnitude = np.abs(fft_result) / n
    return freqs, magnitude

# Analyze tracking error spectrum for problematic joints
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Pick interesting joints
joints_to_analyze = [
    (3, "Left Knee"),
    (15, "Left Shoulder Pitch"),
    (0, "Left Hip Pitch"),
    (12, "Waist Yaw")
]

for ax, (idx, name) in zip(axes.flat, joints_to_analyze):
    freqs, mag = compute_fft(tracking_error[:, idx])
    ax.semilogy(freqs, mag)
    ax.set_title(f'{name} - Tracking Error Spectrum')
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Magnitude')
    ax.set_xlim([0, CONTROL_FREQ/2])
    
    # Find dominant frequency
    peak_idx = np.argmax(mag[1:]) + 1  # Skip DC
    ax.axvline(x=freqs[peak_idx], color='r', linestyle='--', alpha=0.5)
    ax.text(freqs[peak_idx], mag[peak_idx], f' {freqs[peak_idx]:.1f} Hz', color='r')

plt.tight_layout()
plt.show()

In [ ]:
# Torque spectrum analysis (detect oscillations in motor commands)
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

for ax, (idx, name) in zip(axes.flat, joints_to_analyze):
    freqs, mag = compute_fft(torque[:, idx])
    ax.semilogy(freqs, mag)
    ax.set_title(f'{name} - Torque Spectrum')
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Magnitude')
    ax.set_xlim([0, CONTROL_FREQ/2])

plt.tight_layout()
plt.show()

## 9. Phase Space Analysis

Position vs velocity plots can reveal limit cycles and stability characteristics.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(14, 8))

phase_joints = [0, 3, 6, 9, 15, 22]  # Hip and knee joints, shoulders

for ax, idx in zip(axes.flat, phase_joints):
    # Color by time
    scatter = ax.scatter(joint_pos[:, idx], joint_vel[:, idx], 
                        c=time, cmap='viridis', s=1, alpha=0.5)
    ax.set_xlabel('Position (rad)')
    ax.set_ylabel('Velocity (rad/s)')
    ax.set_title(f'{JOINT_NAMES[idx]} Phase Space')
    
plt.colorbar(scatter, ax=axes.flat[-1], label='Time (s)')
plt.tight_layout()
plt.show()

## 10. Correlation Analysis

Check for unexpected correlations between joints (coupling issues).

In [ ]:
# Correlation matrix of tracking errors
error_corr = np.corrcoef(tracking_error.T)

fig, ax = plt.subplots(figsize=(12, 10))
im = ax.imshow(error_corr, cmap='RdBu_r', vmin=-1, vmax=1)
ax.set_xticks(range(len(JOINT_NAMES)))
ax.set_yticks(range(len(JOINT_NAMES)))
ax.set_xticklabels(JOINT_NAMES, rotation=90, fontsize=8)
ax.set_yticklabels(JOINT_NAMES, fontsize=8)
ax.set_title('Tracking Error Correlation Matrix')
plt.colorbar(im, ax=ax, label='Correlation')
plt.tight_layout()
plt.show()

# Find strong correlations (excluding diagonal)
corr_upper = np.triu(error_corr, k=1)
strong_corr = np.where(np.abs(corr_upper) > 0.7)
if len(strong_corr[0]) > 0:
    print("\nStrongly correlated joint errors (|r| > 0.7):")
    for i, j in zip(strong_corr[0], strong_corr[1]):
        print(f"  {JOINT_NAMES[i]} <-> {JOINT_NAMES[j]}: r={error_corr[i,j]:.3f}")

## 11. Interactive Joint Explorer

Select a joint to view detailed analysis.

In [ ]:
def detailed_joint_analysis(joint_idx):
    """Comprehensive analysis for a single joint."""
    joint_name = JOINT_NAMES[joint_idx]
    
    fig = plt.figure(figsize=(14, 10))
    gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)
    
    # Position tracking
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(time, target[:, joint_idx], 'b-', label='Target', alpha=0.7)
    ax1.plot(time, joint_pos[:, joint_idx], 'r-', label='Actual', alpha=0.7)
    ax1.set_title(f'{joint_name} - Position Tracking')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Position (rad)')
    ax1.legend()
    
    # Tracking error
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.plot(time, tracking_error[:, joint_idx], 'g-')
    ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax2.set_title(f'{joint_name} - Tracking Error')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Error (rad)')
    
    # Velocity
    ax3 = fig.add_subplot(gs[1, 0])
    ax3.plot(time, joint_vel[:, joint_idx], 'purple')
    ax3.set_title(f'{joint_name} - Velocity')
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Velocity (rad/s)')
    
    # Torque
    ax4 = fig.add_subplot(gs[1, 1])
    ax4.plot(time, torque[:, joint_idx], 'orange')
    ax4.set_title(f'{joint_name} - Motor Torque')
    ax4.set_xlabel('Time (s)')
    ax4.set_ylabel('Torque (Nm)')
    
    # Phase space
    ax5 = fig.add_subplot(gs[2, 0])
    ax5.scatter(joint_pos[:, joint_idx], joint_vel[:, joint_idx], 
               c=time, cmap='viridis', s=2, alpha=0.5)
    ax5.set_title(f'{joint_name} - Phase Space')
    ax5.set_xlabel('Position (rad)')
    ax5.set_ylabel('Velocity (rad/s)')
    
    # FFT of error
    ax6 = fig.add_subplot(gs[2, 1])
    freqs, mag = compute_fft(tracking_error[:, joint_idx])
    ax6.semilogy(freqs, mag)
    ax6.set_title(f'{joint_name} - Error Spectrum')
    ax6.set_xlabel('Frequency (Hz)')
    ax6.set_ylabel('Magnitude')
    ax6.set_xlim([0, CONTROL_FREQ/2])
    
    plt.suptitle(f'Detailed Analysis: {joint_name}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"\n=== {joint_name} Statistics ===")
    print(f"Position: mean={np.mean(joint_pos[:, joint_idx]):.4f}, std={np.std(joint_pos[:, joint_idx]):.4f}")
    print(f"Tracking Error: RMS={np.sqrt(np.mean(tracking_error[:, joint_idx]**2)):.4f}, max={np.max(np.abs(tracking_error[:, joint_idx])):.4f}")
    print(f"Velocity: mean={np.mean(joint_vel[:, joint_idx]):.4f}, std={np.std(joint_vel[:, joint_idx]):.4f}")
    print(f"Torque: mean={np.mean(torque[:, joint_idx]):.2f}, max={np.max(np.abs(torque[:, joint_idx])):.2f}")

# Example: analyze left knee
detailed_joint_analysis(3)  # left_knee

In [ ]:
# Analyze another joint - change the index to explore different joints
# Joint indices: 0-5 (left leg), 6-11 (right leg), 12-14 (waist), 15-21 (left arm), 22-28 (right arm)
detailed_joint_analysis(15)  # left_shoulder_pitch

## 12. Summary Report

In [ ]:
print("="*60)
print("SIM2REAL ANALYSIS SUMMARY")
print("="*60)

# Overall tracking performance
overall_rms = np.sqrt(np.mean(tracking_error**2))
print(f"\n[TRACKING]")
print(f"  Overall RMS error: {overall_rms:.4f} rad ({np.degrees(overall_rms):.2f} deg)")
print(f"  Worst joint: {JOINT_NAMES[np.argmax(rms_error)]} (RMS={rms_error.max():.4f} rad)")
print(f"  Best joint: {JOINT_NAMES[np.argmin(rms_error)]} (RMS={rms_error.min():.4f} rad)")

# Torque analysis
print(f"\n[TORQUE]")
print(f"  Max torque observed: {np.max(np.abs(torque)):.2f} Nm ({JOINT_NAMES[np.argmax(torque_abs_max)]})")
print(f"  Average power usage: proportional to sum(|torque * velocity|)")

# Stability indicators
print(f"\n[STABILITY]")
print(f"  Quaternion norm deviation: {np.std(quat_norm):.6f}")
print(f"  Max gyro magnitude: {np.max(np.linalg.norm(gyro, axis=1)):.2f} rad/s")

# Warnings
print(f"\n[WARNINGS]")
warnings_found = False

if np.any(rms_error > 0.1):
    warnings_found = True
    print(f"  - High tracking error (>0.1 rad) on: {[JOINT_NAMES[i] for i in np.where(rms_error > 0.1)[0]]}")

if np.std(quat_norm) > 0.01:
    warnings_found = True
    print(f"  - Quaternion normalization issues detected")

if np.max(np.abs(gyro)) > 5.0:
    warnings_found = True
    print(f"  - High angular velocities detected (possible instability)")

if not warnings_found:
    print("  None - system appears stable")

print("\n" + "="*60)